In [1]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Importing the proteins data
ns2a = []
ns2ad = []
        
f = open("../../../../../../Data/Proteins/DENV2/NS2A/DENV2_NS2A.txt", "r")
for x in f:
    if "DSS" in x:
        ns2ad.append(1)
    elif "DHF" in x:
        ns2ad.append(1)
    elif x[0] == ">":
        ns2ad.append(0)
    else:
        x = x.replace("\n", "")
        ns2a.append(x)

In [3]:
# Converting the arrays into DataFrames
ns2a = pd.DataFrame(ns2a)

# Attaching the "Disease" label column to the input
ns2a["Disease"] = ns2ad

# Renaming the columns
ns2a = ns2a.rename(index=str, columns={0: "Sequence", "Disease": "Disease"})

In [4]:
# Concatinating the input data into one DataFrame
print("The combined shape of the given data is:", str(ns2a.shape))
print("The length of the combined data is:", str(len(ns2a.index)))
print("Does the combined data have any null value? ->", ns2a.isnull().values.any())

# ns2a = result.drop(columns = [0])
# ns2a = result.dropna(how='any',axis=0) 

# Shuffling the data and then taking a peek
ns2a = ns2a.sample(frac = 1)
print(ns2a.head())

The combined shape of the given data is: (603, 2)
The length of the combined data is: 603
Does the combined data have any null value? -> False
                                              Sequence  Disease
406  GHGQIDNFSLGVLGMALFLEEMLRTRVGTKHAILLVAVSFVTLITG...        1
254  GHGQIDNFSLGVLGMALLLEEMLRTRIGTKHAILLVAVSFVTLITG...        0
360  GHGQVDNFSLGVLGMALFLEEMLRTRVGTKHAILLVAVSFVTLITG...        0
459  GHGQIDNFSLGVLGMALFLEEMLRTRVGTKHAILLVTVSFVTLITG...        1
350  GHGQVDNFSLGVLGMALFLEEMLRTRVGTKHAILLVAVSFVTLITG...        0


In [5]:
# Function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
def getKmers(sequence, size = 6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

ns2a['words'] = ns2a.apply(lambda x: getKmers(x['Sequence']), axis=1)
ns2a = ns2a.drop('Sequence', axis=1)

ns2a_texts = list(ns2a['words'])
for item in range(len(ns2a_texts)):
    ns2a_texts[item] = ' '.join(ns2a_texts[item])

In [6]:
# Creating y and printing the shape of it
y = ns2a.iloc[:, 0].values
print("The shape of y is:", y.shape)

The shape of y is: (603,)


In [7]:
# Creating the Bag of Words model using CountVectorizer()
# This is equivalent to k-mer counting
# The n-gram size of 4 was previously determined by testing
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(4,4))
x = cv.fit_transform(ns2a_texts)

# Print the shape of x
print("The shape of x matrix is:", x.shape)

The shape of x matrix is: (603, 1220)


## Nested Cross-Validation

In [8]:
from sklearn.model_selection import cross_val_score
# search for an optimal value of C for Logistic Regression
Tree_list = [10, 100, 1000, 2500, 5000]
# empty list to store scores
RF_scores = []

In [9]:
from sklearn.ensemble import RandomForestClassifier
# 1. we will loop through reasonable values of k
for trees in Tree_list:
    # 2. run RandomForestClassifier with trees trees
    lr = RandomForestClassifier(n_estimators = trees, criterion = 'entropy', random_state = 0)
    # 3. obtain cross_val_score for RandomForestClassifier with trees trees
    scores = cross_val_score(lr, x, y, cv = 10, scoring='accuracy')
    # 4. append mean of scores for trees trees to RF_scores list
    RF_scores.append(scores.mean())
    
    print("The score for", trees, "trees, is:", scores.mean())

The score for 10 trees, is: 0.7725798833009169
The score for 100 trees, is: 0.7761345744188201
The score for 1000 trees, is: 0.7727174215059739
The score for 2500 trees, is: 0.7777465962767435
The score for 5000 trees, is: 0.774356765768269


In [10]:
# search for an optimal value of C for Logistic Regression
Tree_list2 = [2400, 2600]
# empty list to store scores
RF_scores2 = []

In [11]:
# 1. we will loop through reasonable values of k
for trees in Tree_list2:
    # 2. run RandomForestClassifier with trees trees
    lr = RandomForestClassifier(n_estimators = trees, criterion = 'entropy', random_state = 0)
    # 3. obtain cross_val_score for RandomForestClassifier with trees trees
    scores = cross_val_score(lr, x, y, cv = 10, scoring='accuracy')
    # 4. append mean of scores for trees trees to RF_scores list
    RF_scores2.append(scores.mean())
    
    print("The score for", trees, "trees, is:", scores.mean())

The score for 2400 trees, is: 0.7777465962767435
The score for 2600 trees, is: 0.7777465962767435
